In [1]:
push!(LOAD_PATH, joinpath(@__DIR__, "../../lib"));

In [2]:
using BenchmarkTools
using NLopt
using Statistics
using PyPlot

In [3]:
using MSSim
const Opts = MSSim.Optimizers
const SS = MSSim.SegSeq
const SL = MSSim.SymLinear
const Seq = MSSim.Sequence

[ Info: Precompiling MSSim [top-level] (cache misses: include_dependency fsize change (1))


MSSim.Sequence

In [4]:
const radial_modes = [
    2.349490,
    2.397870,
    2.439980,
    2.476530,
    2.507130,
    2.531490,
    2.548960
];
const lamb_dicke_parameters = [
    0.12535185674701685,
    0.12411265750465082,
    0.12305413516480627,
    0.12216335005557813,
    0.12143343371677694,
    0.12086534864431214,
    0.12047167276221905
];
const participation_factors = [
    0.02222009251306859 -0.1722648656984975 0.48939776715759997 -0.678705987944334 0.4893977671575958 -0.17226486569849606 0.022220092513065576
    -0.08507556148734259 0.4120558633416861 -0.5683063560469468 0.0 0.5683063560469427 -0.4120558633416741 0.08507556148733478
    -0.21303936535968387 0.5714074376873661 -0.11990319870608773 -0.4769297472432097 -0.1199031987060997 0.5714074376873872 -0.2130393653596884
    0.39521513624944005 -0.44499802171470876 -0.38181377234109953 0.0 0.38181377234109876 0.4449980217146981 -0.39521513624943017
    0.5579098076201139 -0.031003440845807535 -0.3213345695440472 -0.41114359446049004 -0.3213345695440565 -0.031003440845817597 0.5579098076201152
    -0.5801440725517667 -0.3635749250921281 -0.1767657459105281 0.0 0.176765745910517 0.3635749250921191 0.5801440725517734
    0.37796447300922137 0.37796447300923375 0.37796447300922364 0.3779644730092235 0.37796447300922775 0.37796447300923464 0.37796447300922603
];

In [5]:
function mode_weight!(weights, ion1, ion2)
    for i in 1:7
        weights[i] = participation_factors[i, ion1] * participation_factors[i, ion2] * lamb_dicke_parameters[i]^2
    end
end

const modes = Seq.Modes()
for i in 1:7
    push!(modes, 2π * radial_modes[i], 1)
end

In [6]:
blackman(x) = 0.42 + 0.5 * cospi(x) + 0.08 * cospi(2 * x)

struct BlackmanStartEnd{Ratio} <: Function
end

function (::BlackmanStartEnd{Ratio})(x) where Ratio
    if -Ratio <= x <= Ratio
        return float(one(x))
    elseif x < 0
        x = (x + Ratio) / (1 - Ratio)
    else
        x = (x - Ratio) / (1 - Ratio)
    end
    return blackman(x)
end

In [7]:
struct PreOptObjective{NModes}
end

function (::PreOptObjective{NModes})(vals) where NModes
    dis = vals[1]
    disδ = vals[2]
    area = zero(eltype(vals))
    areaδ = zero(eltype(vals))
    for i in 1:NModes
        areai = abs(vals[2 + i])
        areaδi = abs(vals[2 + NModes + i])
        area += areai
        areaδ += areaδi / areai
    end
    return (dis + disδ / 200) * areaδ / area^2
end

In [8]:
const nsegs = [80, 100, 120, 140, 160, 180, 200]
const compute_bufs_opt = [SL.ComputeBuffer{nseg,Float64}(Val(SS.mask_allδ), Val(SS.mask_allδ)) for nseg in nsegs]
const compute_bufs_plot = [SL.ComputeBuffer{nseg,Float64}(Val(SS.mask_full), Val(SS.mask_full)) for nseg in nsegs];

In [9]:
function get_preobj(modes, buf, ratio=0.6)
    nmodes = length(modes.modes)
    area_args = ntuple(i->(:area, i), nmodes)
    areaδ_args = ntuple(i->(:areaδ, i), nmodes)
    return Seq.Objective(SL.pmask_tfm,
                         ((:dis2, 0), (:disδ2, 0), area_args..., areaδ_args...),
                         Opts.autodiff(PreOptObjective{7}()), modes, buf,
                         freq=Seq.FreqSpec(true, sym=false),
                         amp=Seq.AmpSpec(cb=BlackmanStartEnd{ratio}(), mid_order=-1))
end

const preobjs = [get_preobj(modes, buf, 0.6) for buf in compute_bufs_opt];

In [18]:
struct PreOptimizer{NModes,ObjArgs,Obj}
    preobj::Obj
    tracker::Opts.NLVarTracker
    opt::NLopt.Opt
end
function PreOptimizer(preobj::Obj;
                      τmin=0.1, τmax=3, Ω=0.4, ωmin=2π * 2.39, ωmax=2π * 2.52) where Obj
    nargs = Seq.nparams(preobj)
    tracker = Opts.NLVarTracker(nargs)
    Opts.set_bound!(tracker, preobj.param.τ, τmin, τmax)
    Opts.set_bound!(tracker, preobj.param.Ωbase, Ω, Ω)
    for ω in preobj.param.ωs
        Opts.set_bound!(tracker, ω, ωmin, ωmax)
    end

    nmodes = length(preobj.modes)

    opt = NLopt.Opt(:LD_LBFGS, nargs)
    NLopt.min_objective!(opt, preobj)
    NLopt.maxeval!(opt, 2000)

    area_args = ntuple(i->(:area, i), nmodes)
    areaδ_args = ntuple(i->(:areaδ, i), nmodes)

    o = PreOptimizer{nmodes,((:τ, 0), (:dis2, 0), (:disδ2, 0), area_args..., areaδ_args...),Obj}(preobj, tracker, opt)
    update_bounds!(o)
    return o
end

function update_bounds!(o::PreOptimizer)
    NLopt.lower_bounds!(o.opt, Opts.lower_bounds(o.tracker))
    NLopt.upper_bounds!(o.opt, Opts.upper_bounds(o.tracker))
end

struct SolutionInfo{NModes}
    params::Vector{Float64}
    total_t::Float64
    dis2::Float64
    disδ2::Float64
    area::NTuple{NModes,Float64}
    areaδ::NTuple{NModes,Float64}
end

function compute_one(o::PreOptimizer{NModes,ObjArgs}) where {NModes,ObjArgs}
    obj, params, ret = NLopt.optimize(o.opt, Opts.init_vars!(o.tracker))
    if getfield(NLopt, ret) < 0
        return
    end
    info = o.preobj(Val(ObjArgs), params) do x
        return SolutionInfo{NModes}(params, x[1], x[2], x[3], ntuple(i->x[i + 3], NModes),
                                    ntuple(i->x[i + 3 + NModes], NModes))
    end
    area = 0.0
    areaδ = 0.0
    for i in 1:NModes
        areai = abs(info.area[i])
        areaδi = abs(info.areaδ[i])
        area += areai
        areaδ += areaδi / areai
    end
    if abs(info.dis2) < 1e-6 && abs(info.disδ2) < 1e-4 && area >= 15 * NModes
        return info
    end
end

function find_candidates!(o::PreOptimizer, nrounds, candidates=SolutionInfo[])
    for i in 1:nrounds
        info = compute_one(o)
        if info !== nothing
            push!(candidates, info)
        end
    end
    return candidates
end

find_candidates! (generic function with 2 methods)

In [11]:
const preopts = [PreOptimizer(preobj) for preobj in preobjs];

In [21]:
function run_preopts(nseg, o::PreOptimizer, rounds, total_t_min, total_t_max, nsteps)
    res = Vector{SolutionInfo}[]
    for i in 1:nsteps
        τmin = total_t_min + (total_t_max - total_t_min) / nsteps * (i - 1)
        τmax = total_t_min + (total_t_max - total_t_min) / nsteps * i
        println("Testing total time range [$(τmin), $(τmax)]")
        Opts.set_bound!(o.tracker, o.preobj.param.τ, τmin / nseg, τmax / nseg)
        update_bounds!(o)
        @time push!(res, find_candidates!(o, rounds))
        println("Found $(length(res[end])) candidates")
    end
    return res
end

run_preopts (generic function with 1 method)

In [22]:
run_preopts(nsegs[3], preopts[3], 1000, 150, 350, 25)

Testing total time range [150.0, 158.0]
 38.489964 seconds (5.26 M allocations: 1.480 GiB, 0.36% gc time)
Found 441 candidates
Testing total time range [158.0, 166.0]
 45.302380 seconds (6.25 M allocations: 1.759 GiB, 0.35% gc time)
Found 375 candidates
Testing total time range [166.0, 174.0]
 29.662061 seconds (4.01 M allocations: 1.129 GiB, 0.35% gc time)
Found 549 candidates
Testing total time range [174.0, 182.0]
 18.870351 seconds (2.48 M allocations: 715.053 MiB, 0.34% gc time)
Found 501 candidates
Testing total time range [182.0, 190.0]
 19.997569 seconds (2.62 M allocations: 756.565 MiB, 0.34% gc time)
Found 481 candidates
Testing total time range [190.0, 198.0]
 21.081055 seconds (2.77 M allocations: 799.007 MiB, 0.34% gc time)
Found 487 candidates
Testing total time range [198.0, 206.0]
 21.522050 seconds (2.85 M allocations: 822.069 MiB, 0.34% gc time)
Found 465 candidates
Testing total time range [206.0, 214.0]
 20.916675 seconds (2.75 M allocations: 793.932 MiB, 0.34% gc t

25-element Vector{Vector{SolutionInfo}}:
 [SolutionInfo{7}([1.3157692636375973, 0.4, 15.01756204157502, 15.32597391212577, 15.596240557284283, 15.271425450187554, 15.522784434670717, 15.645177237524138, 15.692846002648139, 15.711719728651616  …  15.833626974092557, 15.833626974092557, 15.833626974092557, 15.601765112902568, 15.833626974092557, 15.188948161365328, 15.635814560942881, 15.17081281300836, 15.157396001521473, 15.345203178420954], 157.89231163651166, 1.303564822343776e-11, 8.310740401486123e-11, (28.497744959288216, 47.149807274360285, 100.73617184278451, -104.33210899638664, -64.64390373357143, -40.74032941321454, -30.890813660645303), (-144.035425048936, -379.3578498175252, -1446.0759589930012, -3082.8188934666127, -580.7505719191095, -136.576581364565, -81.27082485555852)), SolutionInfo{7}([1.3166666666666667, 0.4, 15.173539844368495, 15.517044509684535, 15.177593348334918, 15.445614007753273, 15.382452684137892, 15.684856088614964, 15.833626974092557, 15.382883658781521 